# Machine Learning in Julia - MLJ.jl

## Loading the dataset

In [ ]:
using MLJ
using CSV, DataFrames
using Random
using Plots

In [ ]:
df = CSV.read("bank-additional-full.csv", DataFrame, delim=";")

`MLJ` use the notion of the **scientific types** beside the data types. Scientific types specify how the features should be interpreted by the ML models and the users - the basic types are:
* `Continuous`,
* `Count`,
* `Multiclass{N}`, 
* `OrderedFactor{N}`, 
* `Textual`.

More information about the scientific types can be found in the [MLJ documentation](https://alan-turing-institute.github.io/MLJ.jl/dev/getting_started/#Data-containers-and-scientific-types). The `schema` function will produce a data type and scientific type for each column in the dataset.

In [ ]:
schema(df)

The default mapping between the data and scientific types didn't produce the desired outcome. Let's use `coerce` function to change all `Textual` types to `Multiclass` and `age` feature to the `Continuous` type. The binary target variable `y` is also coerced to `OrderedFactor` which will indicate `yes` value to be intepreted as positive class during classification. Note that after the conversion data types changed as well.

In [ ]:
df = coerce(df, :age => Continuous, Textual => Multiclass);
df = coerce(df, :y => OrderedFactor)
schema(df)

In [ ]:
levels!(df.y, ["no","yes"]) # setting the ordering of the `y` column
levels(df.y)

## 3. Data preprocsessing

After modifying the schema, let's proceed with further preprocessing. `unpack` function splits the dataset into independent variables (`X`) and target variable (`y`). Features can also be removed in the same step - let's remove the `duration` variable as it's highly related to the target and recommended for removal by the dataset owner (for details check the `bank-additional-names.txt`).

In [ ]:
combine(groupby(df, :y), :duration .=> [minimum median mean])

Statistics calculated above confirm close relation between target variable and duration of the call - mean `duration` is significantly higher, when customer subscribed to the term deposit. Let's proceed with splitting the dataset and removal of the `duration` variable.

In [ ]:
y, X = unpack(df, ==(:y), !=(:duration));

After the split, `y` is a `CategoricalArray` with two levels: "no" and "yes".

In [ ]:
y

`X` remains a `DataFrame` with `duration` and `y` columns gone.

In [ ]:
X

In most cases data needs to be preprocessed to be properly used for ML models training. `MLJ` has multiple transformers to help with common data preparation tasks, in particular:
* standardizing the numeric features,
* one-hot encoding the nominal features,
* transforming skewed numeric features,
* imputing the missing values.

Let's use `Standardizer` and `OneHotEncoder` transformers on the Bank Marketing data - both [standardization](https://en.wikipedia.org/wiki/Standard_score) and [one-hot encoding](https://en.wikipedia.org/wiki/One-hot#Machine_learning_and_statistics) are common preprocessing steps. By default `Standardizer` standardize only features with `Continuous` scientific type. Similarly, `OneHotEncoder` transforms only `Multiclass` and `OrderedFactor` types. Multiple preparation steps can be combined in **pipelines** by using the `Pipeline` constructor or `|>` symbol - pipelines make repetitive tasks simpler and easier to execute.

`MLJ` expose a common interface for managing and manipulating transformers, models and other objects. To run the preprocessing pipeline we'll utilize:
* `machine` function which binds an algorithm (model, transformer, pipelines, etc.) to the data,
* `fit!` function for training the algorithm,
* `transform` function to use trained pipepline to get the standardized and one-hot encoded dataset.

The cell below will show a transformed `DataFrame` - look how it differs from the input.

In [ ]:
?Standardizer()

In [ ]:
preproc_pipe = Standardizer() |> OneHotEncoder()
preproc_wrapped = machine(preproc_pipe, X)
X_prepared = MLJ.transform(fit!(preproc_wrapped), X)

Using another element from the common interface - `fitted_params` function - we can inspect the parameters learned by the `Standardizer` while running `fit!`.

In [ ]:
fitted_params(preproc_wrapped).standardizer.features_fit

## 4. MLJ as repository of models

`MLJ` framework also works as a repository and wrapper of ML models built in various Julia packages. Using `models` function, let's check which models will be suited to the posed classification task considering types of predictors and target variable. Additionally, let's find only models fully implemented in Julia.

Before a particular model will be used in the `MLJ` workflow, it must be available in the environment. `load_path` function returns a package need for each model.

In [ ]:
condition = m -> (matching(m, X_prepared, y) && m.is_pure_julia)
models(condition)

In [ ]:
for (model, pkg) in [("RandomForestClassifier", "DecisionTree"),
                     ("DecisionTreeClassifier", "DecisionTree")]
println(load_path(model, pkg=pkg))
end

We can load the models with the `@load` macro. If the model name appears in more than one package, the package name must be explicitly stated.

In [ ]:
Forest = @load RandomForestClassifier pkg="DecisionTree"
Tree = @load DecisionTreeClassifier pkg="DecisionTree"

## 5. Training and evaluating the models

We are ready to train the loaded models - let's streamline the training and evaluation process with `MLJ`. The models with the default hyperparameter values are wrapped in the `evaluate` function which will do the following:
* split the provided data according to the resampling strategy - in the case below the `Holdout` specifies the data split into train/test subsets with 80/20 ratio and random seed equal to 42,
* fit the model on the training data and evaluate it on the test data calculating each measure specified in the `measure` parameter,
* return the evaluation report with relevant information.

In [ ]:
combine(groupby(df, :y), nrow=>:count)

How the training and evaluation steps are done step-by-step.

1. Split the data into the training and test dataset

In [ ]:
(X_train, X_test), (y_train, y_test) = partition((X_prepared,y), 0.8, rng=42, multi=true);

2. Train the model

In [ ]:
mach = machine(Forest(), X_train, y_train)
fit!(mach)

3. Make a prediction on the test data

In [ ]:
ŷ = predict_mode(mach, X_test);

4. Evaluate or assess the model

In [ ]:
accuracy(ŷ, y_test)

The same steps can be performed with `evaluate` function.

In [ ]:
measures = [accuracy, auc, f1score]
split = Holdout(fraction_train=0.8, rng=42)
rng_mt = MersenneTwister(42)
for m in [Forest, Tree]
    eval_report = evaluate(m(rng=rng_mt), X_prepared, y, resampling=split, measure=measures)
    println(m)
    println.(eval_report.measure, ": ", round.(eval_report.measurement, digits=3))
    println()
end

To list all available metrics, use `measures` function similar to the `models`. 

In [ ]:
MLJ.measures()

Three ML models with the default hyperparameters values were built and evaluated. Out of the trained models `RandomForestClassifier` achieved the best AUC and F1-score. Let's tune the hyperparameters of the Random Forest  based on the AUC metric. 

The list of available hyperparemeters and their default values is displayed when the instance of the Random Forest is called. 

In [ ]:
forest = Forest(rng=rng_mt)

To learn more about each hyperparameter review documentation for `RandomForestClassifier` by calling `?Forest()`. 

In [ ]:
?Forest()

## 6. Cross-validation and hyperparameter tuning

Tuning all hyperparameters of `RandomForestClassifier` would be computionally demanding. To finish within a reasonable time, we'll tune two arbitrary picked hyperparameters: `max_depth` and `n_trees`. `max_depth` limits the growth of each tree in the ensemble - deep trees may be overfitted, while shallow trees may be biased. `n_trees` determines number of trees in the ensemble - the predictive power of the forest should initially increase with the number of trees and then saturate.

Tuning in `MLJ` can be performed easily with `TunedModel` interface. The tuning specification includes:
* `model` - the model to be tuned, in our case an instance of `RandomForestClassifier`,
* `resampling` - mechanism for splitting the data into training and validation subsets, e.g. cross-validation (below 3-fold cross-validation specified in `CV` resampling strategy),
* `tuning` - tuning strategy for searching the hyperparameters space, in the example below `Grid(resolution=4)` constructs grid search with cartesian product of six evenly spaced values for each tuned hyperparameter (16 models for two tuned hyperparameters),
* `range` - specification of tuned hyperparameters including hyperparameter names and their extreme values,
* `measure` - metric used for models evaluation - as specfied above, AUC is used.

Please note the tuning is computationally intensive excercise and may take few minutes to complete. The progress bar will indicate how many models are already evaluated.

In [ ]:
hyperparam_range = [range(forest, :n_trees, lower=10, upper=80),
                    range(forest, :max_depth, lower=2, upper=30)]
self_tuning_forest = TunedModel(
    model = forest,
    resampling = CV(nfolds=3, rng=rng_mt),
    tuning = Grid(resolution=4),
    range = hyperparam_range,
    measure = auc)
mach = machine(self_tuning_forest, X_prepared, y)
fit!(mach, verbosity=1)

In [ ]:
plot(mach)

In [ ]:
best_model = fitted_params(mach).best_model

Using initial `evaluate` call we can easily compare tuned model performance to the default forest.

In [ ]:
evaluate(best_model, X_prepared, y, resampling=split, measure=measures)

In [ ]:
predict_mode(mach, X_prepared[1:10,:])